In [22]:
import sqlite3
import json

import sys
sys.path.append("../../05_src/")

In [23]:
def sanitize_string(s):
    if isinstance(s, str):
        s = s.encode('utf-8', errors='ignore').decode('utf-8', errors='ignore')
        s = s.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        s = s.replace('\u0720', ' ')
        s = s.replace("\n", " ")
    return s


def get_data_from_sqlite(db_path, table):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()
        columns = [description[0] for description in cursor.description]
        data = [dict(zip(columns, row)) for row in rows]
        data = [ {k: sanitize_string(v) for k, v in record.items()} for record in data ]
    return data

In [24]:
def create_jsonl_from_table(db_path, table):
    data = get_data_from_sqlite(db_path, table)
    with open(f'../../05_src/documents/pitchfork_{table}.jsonl', 'w') as f:
        for record in data:    
            json_line = json.dumps(record)
            f.write(json_line + '\n')

In [25]:
sqlite_file = "../../05_src/documents/database.sqlite"


In [26]:
table_list = ['artists', 'content', 'genres', 'labels', 'reviews', 'years']

for table in table_list:
    create_jsonl_from_table(sqlite_file, table)